In [1]:
import numpy as np
from itertools import product
from glob import glob
from tqdm.auto import tqdm
import gzip
import pandas as pd
import re
from joblib import Parallel, delayed

In [2]:
cd C:/Users/parashar/Desktop/ucsc

C:\Users\parashar\Desktop\ucsc


In [3]:
def meme_reader(fn):
    a = []
    with open(fn) as h:
        while True:
            l = next(h)
            if l.startswith('ALPHABET'):
                base_map = dict(enumerate(l.split('= ')[1].rstrip('\n')))
                assert len(base_map) == 4
            elif l.startswith('MOTIF'):
                m_name = l.rstrip('\n').split(' ')[-1]
            elif l.startswith('letter-probability'):
                break
        for l in h:
            if l.startswith('URL'):
                break
            a.append(l.strip().split('  '))
    a = np.array(a).astype(float)
    return m_name.upper(), a, base_map

def make_sequences(pwm, base_map, meme_fn, min_base_prob=0.25, top_n=10):
    bases = []
    for n,i in enumerate(pwm > min_base_prob):
        pos = list(np.where(i)[0])
        if len(pos) == 0:
            bases.append([pwm[n].argmax()])
        else:
            bases.append(list(np.where(i)[0]))
    combinations = np.array([list(x) for x in product(*bases)])
    comb_values = np.choose(combinations, pwm.T).sum(axis=1)/combinations.shape[1]
    idx = comb_values.argsort()[-10:]
    seq = np.array([list(map(base_map.get,x)) for x in combinations[idx]])
    return seq, comb_values[idx]

def make_pattern(seq_matrix):
    assert len(seq_matrix) > 0
    p = ""
    for i in range(seq_matrix.shape[1]):
        s = list(set(seq_matrix[:, i]))
        if len(s) == 1:
            p += s[0]
        else:
            p += f"[{''.join(s)}]"
    return p

def prep_motifs(meme_dir):
    ret_val = {}
    for meme_fn in tqdm(sorted(glob(f"{meme_dir}/*.meme"))):
        m_name, pwm, base_map = meme_reader(meme_fn)
        a, b = make_sequences(pwm, base_map, meme_fn)
        pat = make_pattern(a)
        if m_name in ret_val:
            print (f"WARNING: Duplicate name: {m_name}")
        ret_val[m_name] = {
            'pattern': re.compile(pat),
            'sequences': [''.join(x) for x in a],
            'scores': list(b.round(5))
        }
    return ret_val

def find_motifs(motifs, chrom_seq, chrom_coords, chrom, disable_tqdm=True):
    res = {}
    for i in tqdm(chrom_coords.values, desc=chrom, total=chrom_coords.shape[0], disable=disable_tqdm):
        s = chrom_seq[i[0]:i[1]]
        c_name = f"{chrom}:{i[0]}-{i[1]}"
        res[c_name] = {}
        for m in motifs:
            a = motifs[m]['pattern'].findall(s)
            score = None
            if len(a) > 0:
                a = list(set(a))
                try:
                    score = motifs[m]['scores'][motifs[m]['sequences'].index(a[0])]
                except ValueError:
                    score = None
                if len(a) > 1:
                    for j in a[1:]:
                        try:
                            temp = motifs[m]['scores'][motifs[m]['sequences'].index(j)]
                        except ValueError:
                            pass
                        else:
                            if score is None:
                                score = temp
                            elif temp > score:
                                score = temp
            if score is not None:
                res[c_name][m] = score
    return res

def generate_stream(intervals_fn, chrom_dir):
    coords = pd.read_csv(intervals_fn, sep='\t', header=None)
    chroms = [f"chr{x}" for x in range(1, 23)] + ['chrX', 'chrY']
    res = []
    for chrom in chroms:
        h = gzip.open(f"{chrom_dir}/{chrom}.fa.gz", mode='rt')
        next(h)
        cs = ''.join([x.rstrip('\n').upper() for x in h])
        df = coords[coords[0] == chrom][[1, 2]]
        yield cs, df, chrom

In [4]:
motifs_info = prep_motifs('jaspar')

  0%|          | 0/841 [00:00<?, ?it/s]

In [7]:
res = Parallel(n_jobs=4)(delayed(find_motifs)(motifs_info, i, j, k) for i,j,k in tqdm(generate_stream('C:/Users/parashar/Desktop/ucsc/coord.bed', 'hg38')))
res = {k: v for d in res for k, v in d.items()}

0it [00:00, ?it/s]

In [44]:
peaks = list(res.keys())
features = {x: n for n,x in enumerate(sorted(set([item for sublist in [list(x.keys()) for x in res.values()] for item in sublist])))}

In [34]:
import scipy.sparse as ss

In [58]:
m = ss.lil_matrix((len(peaks), len(features)))
m

In [59]:
for n,i in enumerate(res):
    idx = list(map(features.get, res[i].keys()))
    if len(idx) > 0:
        m[n, idx] = list(res[i].values())

In [64]:
100*m.nnz/(m.shape[0]*m.shape[1])

1.5476779666986773